## Automated ML

Importing all needed dependencies to complete the project.

In [ ]:
import requests
import json
import logging
import joblib
from pprint import pprint
import pandas as pd
from sklearn.model_selection import train_test_split

import azureml.core
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails
from azureml.core.model import Model, InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.automl.core.shared import constants
from azureml.core.environment import Environment

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)


## Workspace

The config.json file is downloaded from Azure environment and has to be in the same folder in order for this cell to run.

In [ ]:
ws = Workspace.from_config()

print("Workspace name: ", ws.name)
print("Subscription id: ", ws.subscription_id)
print("Resource group: ", ws.resource_group, sep='\n')

## Create an Azure ML experiment
I am creating an experiment named "automl_heart_failure_experiment" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

In [ ]:
# choose a name for experiment
experiment_name = "automl_heart_failure_experiment"
project_folder = './heart_failure-project'

experiment = Experiment(ws,experiment_name)

## Create or Attach an AmlCompute cluster
For the Automl run, we need to create a compute target.

In [ ]:
# Choose a name for the cluster
cpu_cluster_name = 'cluster-cpu'

# Verify that cluster does not exist already

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2', min_nodes=1, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

The dataset contains medical records of 299 patients who had heart failure, collected during their follow-up period, where each patient profile has 13 clinical features.

I am using this data in order to predict the DEATH_EVENT i.e. whether or not the patient deceased during the follow-up period (boolean).

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

The dataset we will be using in this project is called Heart failure clinical records Data Set and is publicly available from UCI Machine Learning Repository.

In [ ]:
# test to see if dataset is in store

key = "heart-failure"
description_text = "Heart failure survival prediction"


if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print('The Dataset was found')
else:
    # Create AML Dataset and register it into Workspace
    data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv"
    dataset = Dataset.Tabular.from_delimited_files(data_url)
    #Register Dataset in Workspace
    dataset = dataset.register(workspace = ws,name = key,description = description_text)

df = dataset.to_pandas_dataframe()
    

In [ ]:
# Preview of the first five rows
print(df.head())

# Explore data
print(df.describe())

## AutoML Configuration

"experiment_timeout_minutes": 30 - This is an exit criterion and is used to define how long, in minutes, the experiment should continue to run. To help avoid experiment time out failures, I used the value of 30 minutes.

"enable_early_stopping": True - It defines to enable early termination if the score is not improving in the short term. In this experiment, it could also be omitted because the experiment_timeout_minutes is already defined below.

"primary_metric": 'accuracy' - I chose accuracy as the primary metric as it is the default metric used for classification tasks.

"n_cross_validations": 4 - This parameter sets how many cross validations to perform, based on the same number of folds (number of subsets). Is set to 4, therefore the training and validation sets will be divided into four equal sets.

"max_concurrent_iterations": 4 - It represents the maximum number of iterations that would be executed in parallel.

"verbosity": logging.INFO - The verbosity level for writing to the log file.

task = 'classification' - This defines the experiment type which in this case is classification. Other options are regression and forecasting.

training_data = dataset - the loaded dataset for the project

label_column_name = 'DEATH_EVENT' - The name of the label column i.e. the target column based on which the prediction is done.

featurization = 'auto' - This parameter defines whether featurization step should be done automatically as in this case (auto) or not (off).

debug_log = 'automl_errors.log - The log file to write debug information to.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes" : 30,
    "enable_early_stopping" : True,
    "primary_metric":'accuracy',
    "n_cross_validations":4,
    "max_concurrent_iterations":4,
    "verbosity": logging.INFO
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = compute_target,
                            task = 'classification',
                            training_data=dataset,
                            path = project_folder,
                            label_column_name="DEATH_EVENT",
                            featurization= 'auto',
                            debug_log = "automl_errors.log",
                            **automl_settings
                            )

In [ ]:
# Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)
remote_run.wait_for_completion()

In [ ]:
# get_status()
# Fetch the latest status of the run. It should show 'Completed'

print("Run Status: ",remote_run.get_status())

## Run Details

In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()
remote_run.wait_for_completion (show_output = True)

## Best Model

In the cell below, get the best model from the automl experiments and display all the properties of the model.

In [ ]:
best_run,fitted_model = remote_run.get_output()

In [ ]:
# Best run

print(best_run)

In [ ]:
# get_metrics()
print(best_run.get_metrics())

In [ ]:
# get_details()
print(best_run.get_details())

In [ ]:
# get_properties()
print(best_run.get_properties())

In [ ]:
# Save the best model
best_run.register_model(model_name = 'automl-best-model.pkl',model_path = './outputs/')

In [ ]:
best_run.get_file_names()

# Download the yaml file that includes the environment dependencies
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'env.yml')

In [ ]:

# Download the model file

best_run.download_file('outputs/model.pkl', 'Automl_model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Registring the best model
model = remote_run.register_model(model_name='automl-best-model.pkl')
print(remote_run.model_id)

# Get automl environment with its dependencies
environment = Environment.get(ws, "AzureML-AutoML")

environment = best_run.get_environment()
entry_script='inference/scoring.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)

inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)


In [ ]:
# Getting the service state
# The scorig URI & the primary authentication key are copied to the endpoint.py file in order to test the deployed service.
# The Swagger URI can be used in Swagger UI: https://petstore.swagger.io/ For more info, please see the relevant part in the README file.

# Authentication is enabled, so I use the get_keys method to retrieve the primary and secondary authentication keys:
primary, secondary = service.get_keys()

print('Service state: ' + service.state)
print('Service scoring URI: ' + service.scoring_uri)
print('Service Swagger URI: ' + service.swagger_uri)
print('Service primary authentication key: ' + primary)


In the cell below, send a request to the web service you deployed to test it.

In [ ]:
%run endpoint.py

import requests
import json

# URL for the web service, should be similar to:

scoring_uri = ''

# If the service is authenticated, set the key or token

key = ''

data = {"data":
        [
          {
           "age": 60, 
           "anaemia": 1, 
           "creatinine_phosphokinase": 315, 
           "diabetes": 1, 
           "ejection_fraction": 60, 
           "high_blood_pressure": 0, 
           "platelets": 454000, 
           "serum_creatinine": 1.1, 
           "serum_sodium": 131, 
           "sex": 1, 
           "smoking": 1,
           "time": 10
          },
          {
           "age": 55, 
           "anaemia": 0, 
           "creatinine_phosphokinase": 1820, 
           "diabetes": 0, 
           "ejection_fraction": 38, 
           "high_blood_pressure": 0, 
           "platelets": 270000, 
           "serum_creatinine": 1.2, 
           "serum_sodium": 139, 
           "sex": 0, 
           "smoking": 0,
           "time": 271
          },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'


# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())
print("Expected result: [true, false], where 'true' means '1' and 'false' means '0' as result in the 'DEATH_EVENT' column")

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# Printing the logs
print(service.get_logs())

## Deleting the service
Putting the deletion of the service in a separate cell to avoid accidentally running the cell before finishing the tasks

In [ ]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
